**Train a simple deep CNN on the CIFAR10 small images dataset**
- It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
- It's still underfitting at that point, though.

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

D:\ProgramData\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)

In [5]:
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 strides=(1,1),padding='same',
                 activation='relu',input_shape=(32,32,3)))
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 strides=(1,1),padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64,kernel_size=(3,3),
                 strides=(1,1),padding='same',
                 activation='relu'))
model.add(Conv2D(filters=64,kernel_size=(3,3),
                 strides=(1,1),padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(units=512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=num_classes,activation='softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
__________

In [6]:
x_train = x_train.astype('float32')/255.
x_test = x_test.astype('float32')/255.

In [7]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train,y_train,
              batch_size=batch_size,epochs=epochs,verbose=1,
              validation_data=(x_test,y_test),shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    model.fit_generator(datagen.flow(x_train,y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),verbose=1)
    # save model and weights
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir,model_name)
    model.save(model_path)
    print('Saved trained model at %s'%model_path)
    
    # score trained model.
    scores = model.evaluate(x_test,y_test,batch_size=batch_size,verbose=1)
    print('Test loss:',scores[0])
    print('Test accuracy:',scores[1])

Using real-time data augmentation.
Epoch 1/10
1563/1563 [==============================] - 59s 38ms/step - loss: 1.8459 - acc: 0.3258 - val_loss: 1.5677 - val_acc: 0.4405
Epoch 2/10
1563/1563 [==============================] - 57s 36ms/step - loss: 1.5541 - acc: 0.4348 - val_loss: 1.3598 - val_acc: 0.5162
Epoch 3/10
1563/1563 [==============================] - 61s 39ms/step - loss: 1.4343 - acc: 0.4824 - val_loss: 1.3350 - val_acc: 0.5216
Epoch 4/10
1563/1563 [==============================] - 60s 38ms/step - loss: 1.3394 - acc: 0.5199 - val_loss: 1.2517 - val_acc: 0.5579
Epoch 5/10
1563/1563 [==============================] - 61s 39ms/step - loss: 1.2678 - acc: 0.5488 - val_loss: 1.1200 - val_acc: 0.6065
Epoch 6/10
1563/1563 [==============================] - 60s 39ms/step - loss: 1.2059 - acc: 0.5704 - val_loss: 1.0988 - val_acc: 0.6102
Epoch 7/10
1563/1563 [==============================] - 60s 38ms/step - loss: 1.1595 - acc: 0.5871 - val_loss: 1.0248 - val_acc: 0.6347
Epoch 8/10
15